In [135]:
import rasterio
import numpy as np
import pandas as pd
import fiona
import time
from shapely.geometry import mapping, shape
from shapely.geometry import box as shBox
from rasterio import windows as win

In [136]:
# set the year
t1=time.time()

# 2016

In [137]:
# set the address of nlcd products here
year = 2016
nlcd_tc_Address=r'G:\NationalLayer\NLCD_Products\2016\nlcd_2016_treecanopy_2019_08_31\nlcd_2016_treecanopy_2019_08_31_projected.img'
nlcd_lc_Address=r'G:\NationalLayer\NLCD_Products\2016\NLCD_2016_Land_Cover_L48_20190424\NLCD_2016_Land_Cover_L48_20190424_projected.img'

# 2011

In [75]:
year = 2011
nlcd_tc_Address=r'G:\NationalLayer\NLCD_Products\2011\nlcd_2011_treecanopy_2019_08_31\nlcd_2011_treecanopy_2019_08_31_Projected.img'
nlcd_lc_Address=r'G:\NationalLayer\NLCD_Products\2011\NLCD_2011_Land_Cover_L48_20190424\NLCD_2011_Land_Cover_L48_20190424_projected.img'

In [138]:
# read processed csv file for climate data. you can use the processing script to prepare this csv file
df_storm=pd.read_csv(r'G:\NationalLayer\ClimateData\DailySummary\dfUS{}_processed.csv'.format(year))

C:\ProgramData\Anaconda2\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [150]:
# set the address of station shapefile and its allocation raster address here
stationPointShape=r'G:\NationalLayer\ClimateData\Stations\Stations_All_USAl.shp'
stationAllocationAddress=r'G:\NationalLayer\ClimateData\Stations\Allocation_StationFIDs30.img'
nlud_Address= r'G:\NationalLayer\NationalLandUse_0140326\nlud2010_Projected.img'
# set the ecosystem accounting area address here
#ecoAcAreasShapefile = r'D:\temp\UrbanEA_HighRes\watershedDenver\City_Watershed.shp'
ecoAcAreasShapefile = r'G:\NationalLayer\Census\UrbanAreas\nhgis0054_shape_Places_2010_withPop_hsng\US_place_2010_Projected_50k_pop.shp' 
ecoAcAreasRaster=r'G:\NationalLayer\Census\UrbanAreas\nhgis0054_shape_Places_2010_withPop_hsng\Places_50k_30m_raster.img'

# set the address of the LAI rasters for summer and winter time
LAI_LeafOnAddress =r'D:\temp\LeafAreaIndex\Leaf_on2_30m_Projected.img'
LAI_LeafOffAddress=r'D:\temp\LeafAreaIndex\Leaf_off2_30m_Projected.img'

# set the fields to get name and IDs
nameField='NAMELSAD10'
id_field='Pl_Uniq_id'
cso_field='CSO_StormW'

In [151]:
# get the list of codes and the list of names
lcCodes=[0,11,12,21,22,23,24,31,41,42,43,51,52,71,72,73,74,81,82,90, 95]
lcNames=['','Open Water','Perennial Ice/Snow','Developed Open Sapce','Developed Low Intensity',
         'Developed Medium Intensity','Developed High Intensity','Barren Land','Deciduous Forest','Evergreen Forest',
         'Mixed Forest','Dwarf','Shrub','Grassland','Sedge/Herb','Lichens','Moss','Pasture/hay','Cultivated Crops',
        'Woody Wetlands','Emergent Herbaceous Wetlands'
        ]
# get the list of codes and the list of names
LUcodes=[0,111,112,113,121,131,132,141,151,161,171,172,211,212,213,214,215,
       221,222,223,231,233,234,241,242,243,244,245,246,249,251,252,255,
       261,310,311,312,313,314,315,321,330,341,342,410,411,412,413,415,
       417,418,419,421,422,423,511,512,513,514,515,516,517,518,
       519,521,522,523,531,532
      ]

LUnames=['','Water Nat. Lake','Water Nat. Swamp/marsh','Water Nat. Playa','Water Hum. Reservoir','Water River',
      'Water Wash','Water Estuary','Water Canal/ditch','Water Wetlands','Water Open ocean','Water Bay/inlet',
       'BU Res. dense urba (>0.1 ac)','BU Res. Urban (0.1 - 1 ac)','BU Res. Suburban (1 - 2.5 ac)','BU Res. Exurban (2.5 - 10 ac)','BU Res. Rural (10 - 40 ac)',
       'BU Comm. Office','BU Comm. Retail/shopping','BU Comm. Entertainment','BU Ind. Factory, plant','BU Ind. Landfill waste trmt)',
       'BU Ind. Utilities','BU Inst. Schools (developed)','BU Inst. Schools (undeveloped)','BU Inst. Medical','BU Inst. Government/publicl',
       'BU Inst. Military (developed)','BU Inst. Military (undeveloped)','BU Inst. Prison','BU Trans. Airports','BU Trans. Highways/railways',
       'BU Trans. Undeveloped','BU Misc','Prod Cropland (general)','Prod Row crops','Prod Pastureland','Prod orchards','Prod sod & switchgrass',
       'Prod aquaculture','Prod Grazed','Prod Mining (open pit)','Prod Timber','Prod Timber plantations','Rec Dev. undifferentiated',
       'Rec Dev. Urban park','Rec Dev. Golf course','Rec Dev. Motorized','Rec Dev. Resort/Ski area','Rec Dev. Campground',
       'Rec Dev. Picnic area','Rec Dev. Boat/fish access','Rec. Nat. park','Rec. Nat. Recreation Area','Rec. Nat. Scenic Area',
       'Cons. Public Wildlife habitat','Cons. Public Conservation area','Cons. Public Nature Reserve','Cons. Public Wilderness',
       'Cons. Public ACEC/RNA','Cons. Public FWS Refuge','Cons. Public Wilderness study area','Cons. Public Archeaology, historic site',
       'Cons. Public Wild & Scenic River','Cons. Public-limited Municipal Watershed','Cons. Public-limited Corps Engineers',
       'Cons. Marine Protected Area','Cons. Private easement','Cons. Private ag easement'
      ]


# these are NAICS codes. We match the use type to NAICS codes 
dictNicsUses = {0:0,111:1,112:1,113:1,121:1,131:1,132:1,141:1,151:1,161:1,171:1,172:1,211:2,212:2,213:2,214:2,215:2,
       221:5,222:6,223:7,231:8,233:10,234:10,241:11,242:11,243:12,244:13,245:13,246:13,249:13,251:14,252:14,255:1,
       261:1,310:1,311:1,312:1,313:1,314:1,315:1,321:1,330:1,341:1,342:1,410:1,411:1,412:1,413:1,415:1,
       417:1,418:1,419:1,421:1,422:1,423:1,511:1,512:1,513:1,514:1,515:1,516:1,517:1,518:1,
       519:1,521:1,522:1,523:1,531:1,532:1 
               }
def creatDictionary (lstCodes,lstNames):
    dictNames={}
    for i in range(0,len(lstCodes)):
        dictNames[lstCodes[i]]=lstNames[i]
    return dictNames

# create a dictionary for landcover codes and names
dicLcCodes=creatDictionary(lcCodes,lcNames)
# create a dictionary for landuse codes and names
dicLuCodes = creatDictionary(LUcodes,LUnames)

def landCoverCodeConvert(codeNumb):
    return (dicLcCodes[codeNumb])

def landUseCodeConvert(codeNumb):
    return (dicLuCodes[codeNumb])

# this fucntion converts use types to NAICS Codes
def landuseToNAIOCS(useNumber):
    return (dictNicsUses[useNumber])

In [152]:
def findWindow (shapeBound,mainRasterBnd,mainRasterCellSize):
    startRow = int((mainRasterBnd[3] - shapeBound[3])/mainRasterCellSize)
    endRow   = int((shapeBound[3] - shapeBound[1])/mainRasterCellSize)+1+startRow
    startCol = int((shapeBound[0] - mainRasterBnd[0])/mainRasterCellSize)
    endCol   = int((shapeBound[2] - shapeBound[0])/mainRasterCellSize)+1+startCol
    return (startRow,endRow,startCol,endCol)

In [153]:
df_storm.columns

Index(['Unnamed: 0', 'Date', 'PRCP_Type', 'prcp', 'Measurement_Flag',
       'Quality_Flag', 'Source_Flag', 'col99', 'Ycoor', 'Xcoor', 'Elevation',
       'State', 'Place', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8',
       'col9', 'col10', 'SOS', 'EOS', 'st_unique_ID', 'NOAA_st_ID', 'leaf_on'],
      dtype='object')

In [154]:
df_storm.sample(2)

,Unnamed: 0,Date,PRCP_Type,prcp,Measurement_Flag,Quality_Flag,Source_Flag,col99,Ycoor,Xcoor,...,col6,col7,col8,col9,col10,SOS,EOS,st_unique_ID,NOAA_st_ID,leaf_on
1461695,USC00153025,2016-03-31,PRCP,0.0015,NaN,NaN,7,600.0,38.2197,-84.8336,...,0.0,NaN,NaN,NaN,NaN,2016-03-29,2016-10-20,39371,USC00153025,1
237678,US1FLHL0010,2016-04-16,PRCP,0.0008,NaN,NaN,N,NaN,27.6005,-81.5093,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,8241,US1FLHL0010,0


In [155]:
with rasterio.open(nlcd_tc_Address) as rst_nlcd_tc:
            kwds = rst_nlcd_tc.meta.copy()
            mainRasterBnd=rst_nlcd_tc.bounds
            cellSize= kwds['transform'][0]

In [156]:
def calInterception(LAI_ar,storm,nlcd_tc):
    # calculate maximum storage capaicty (saturation) ===> Svmax=LAI * 0.0002 (Wang, 2008)
    maxStorage =LAI_ar * nlcd_tc *0.0002
    # we use np.multiply to put together precip events to cells. c=np.array([1,2,3,4,5,6]) p=np.array([[10],[20],[30],[40]]) 
    ## np.multiply(p,c) np.sum(np.multiply(p,c),axis=0)
    storm = storm.reshape(storm.shape[0],1)
    # first we need to create 1-d nlcd-tc; we can convert them to 2-d at the end of procedure. 
    nlcd_tc_1d=nlcd_tc.flatten()
    # The equation now is pc = p-p*e^(-kLAI) where K is 0.7
    ## let's calculate te second part first P*e^(-kLAI) and name it R=p*e^(-kLAI) 
    eKLAI= np.exp(-1*0.7*LAI_ar.flatten())
    # now matrices_multiply(P X eKLAI)
    R = np.multiply(storm,eKLAI)
    # now P-R
    pi= storm-R
    # so far this was for 1m only. Now we multiply by TC area
    pi_tcArea= pi*nlcd_tc.flatten()
    # pi_leaf matrices have one row for each storm and each col is the pixels, now for each event each pixel we compare it with the Smax and cap the interception at Smax
    pi_capped= np.minimum(pi_tcArea,maxStorage.flatten())
    # now sum up all events for the cell. sums of cols
    pi_pixel= np.nansum(pi_capped, axis=0)
    # calculate the rain that falls on the canopy
    canopyRain = np.nansum(np.multiply(storm,nlcd_tc_1d),axis=0)
    # convert them to 2D array again
    pi_pixel2D=pi_pixel.reshape(nlcd_tc.shape)
    canopyRain2D = canopyRain.reshape(nlcd_tc.shape)
    return [pi_pixel2D,canopyRain2D]

In [117]:
eaa_id

10

In [157]:
lstDFsLandcover=[]
lstDFsLanduse =[]
lstErrorEAA=[]

# denver is 265
# NYC is 571
# Seattle is 215
for pol in fiona.open(ecoAcAreasShapefile)[480:481]:
#    try:
    eaa_name=(pol['properties'][nameField])
    GISjoins=(pol['properties']['GISJOIN'])
    eaa_id=(pol['properties'][id_field])
    CSO_Type=(pol['properties'][cso_field])
    poly=(shape(pol['geometry']))
    #msaPoly=[shape(pol['geometry']) for pol in fiona.open(masShapeAddress)]

    with rasterio.open(nlcd_tc_Address) as rst_nlcd_tc:
        kwds = rst_nlcd_tc.meta.copy()
        mainRasterBnd=rst_nlcd_tc.bounds
        cellSize= kwds['transform'][0]

    polyBound = poly.bounds
    # create a window parameter tuple.   
    winProcessing=findWindow(polyBound,mainRasterBnd,cellSize)
    #(row_start, row_stop), (col_start, col_stop)
    window_use=((winProcessing[0],winProcessing[1]),(winProcessing[2],winProcessing[3]))

    # set the cells that do not have the city id as np.nan. This way we are getting cells insdie the boundary only.
    with rasterio.open(ecoAcAreasRaster) as src:
        eaaAr=src.read(1, window=window_use)
        
    addressNewEaa = r'D:\temp\UrbanEA_HighRes\watershedDenver\eaa_watershed.img'
    with rasterio.open(addressNewEaa) as rsc:
        kwdsCustomEaa = rsc.meta.copy()
        eaaCustomBounds=rsc.bounds
        eaaCustomNodata =kwdsCustomEaa['nodata']
        cellSizeCustom= kwdsCustomEaa['transform'][0]
        customEaaWindow = findWindow(polyBound, eaaCustomBounds,cellSizeCustom)
        customeEaaWindow_use=((customEaaWindow[0],customEaaWindow[1]),(customEaaWindow[2],customEaaWindow[3]))
        eaaAr=rsc.read(1, window=customeEaaWindow_use)

#     # when running high resolution layers for a snigle city use the following block instead of nlcd-tc block
#     addressHighResCity = r'D:\temp\UrbanEA_HighRes\NYC_tree_Cover.img'
#     with rasterio.open(addressHighResCity) as rsc:
#         kwdsHighResCity = rsc.meta.copy()
#         HighResBound=rsc.bounds
#         HighResNoData=kwdsHighResCity['nodata']
#         cellSizeHighRes= kwdsHighResCity['transform'][0]
#         HighResWindow = findWindow(polyBound, HighResBound,cellSizeHighRes)
#         HighResWindow_use=((HighResWindow[0],HighResWindow[1]),(HighResWindow[2],HighResWindow[3]))
#         HighResCityAr=rsc.read(1, window=HighResWindow_use)
#         HighResCityAr=HighResCityAr.astype('float')
#         nlcd_tc_nodata = rsc.nodata
#         HighResCityAr[HighResCityAr==nlcd_tc_nodata]=0
#         nlcd_tc= HighResCityAr*9
#         nlcd_tc[eaaAr!=eaa_id]=np.nan

    with rasterio.open(nlcd_tc_Address) as rst_nlcd_tc:
        nlcd_tc_win_ar=rst_nlcd_tc.read(1, window=window_use)
        nlcd_tc_win_ar=nlcd_tc_win_ar.astype('float')
        # convert percent cover to area of tc
        nlcd_tc= nlcd_tc_win_ar*9
        nlcd_tc[eaaAr!=eaa_id]=np.nan

    with rasterio.open(stationAllocationAddress) as src:
        station_id_ar=src.read(1, window=window_use)
        # here we get the station ids that are only in city boundaries
        station_id_ar=station_id_ar[eaaAr==eaa_id]


    with rasterio.open(nlcd_lc_Address) as src:
        nlcd_lc=src.read(1, window=window_use)
        nlcd_lc=nlcd_lc.astype('float')
        nlcd_lc[eaaAr!=eaa_id]=np.nan

    with rasterio.open(nlud_Address) as src:
        nlud=src.read(1, window=window_use)
        nlud=nlud.astype('float')
        nlud[eaaAr!=eaa_id]=np.nan

    # read the low bounds of LAI from Copernicus data 
    with rasterio.open(LAI_LeafOnAddress) as src:
        LAI_leafOn_ar_low=(src.read(1, window=window_use)).astype('float')
        LAI_leafOn_ar_low[eaaAr!=eaa_id]=np.nan

    with rasterio.open(LAI_LeafOffAddress) as src:
        LAI_leafOff_ar_low=(src.read(1, window=window_use)).astype('float')
        LAI_leafOff_ar_low[eaaAr!=eaa_id]=np.nan

    # let's create an average LAI based on i-Tree
    LAI_leafOn_ar_mean=(nlcd_tc*0+1)*4.9
    LAI_leafOff_ar_mean=(nlcd_tc*0+1)*2.9

    # let's create an upper bound that is 10% higher than the average
    LAI_leafOn_ar_up=(nlcd_tc*0+1)*5.39
    LAI_leafOff_ar_up=(nlcd_tc*0+1)*3.19


    unique, counts = np.unique(station_id_ar.flatten(), return_counts=True)

    dfTemp_leafOn =df_storm[(df_storm.st_unique_ID.isin(unique))&(df_storm.leaf_on==1)&(df_storm.prcp>0)]
    dfTempOn=dfTemp_leafOn.groupby(['Date']).agg({'prcp': ['max']})
    dfTempOn.columns.to_flat_index()
    stormLeafOn=np.array(dfTempOn['prcp']['max'])

    dfTemp_leafOff =df_storm[(df_storm.st_unique_ID.isin(unique))&(df_storm.leaf_on==0)&(df_storm.prcp>0)]
    dfTempOff=dfTemp_leafOff.groupby(['Date']).agg({'prcp': ['max']})
    dfTempOff.columns.to_flat_index()
    stormLeafOff=np.array(dfTempOff['prcp']['max'])


    # now we can use the intercept function to calculate rainfall interception in leaf-on and leaf-off time for low, mean, and upper bounds
    # leaf on
    intercept_leafOn_low,  canRain_leafOn_low  = calInterception(LAI_leafOn_ar_low,stormLeafOn,nlcd_tc)
    intercept_leafOn_mean, canRain_leafOn_mean = calInterception(LAI_leafOn_ar_mean,stormLeafOn,nlcd_tc)
    intercept_leafOn_up,   canRain_leafOn_up   = calInterception(LAI_leafOn_ar_up,stormLeafOn,nlcd_tc)
    # leaf off
    intercept_leafOff_low,  canRain_leafOff_low  = calInterception(LAI_leafOff_ar_low,stormLeafOff,nlcd_tc)
    intercept_leafOff_mean, canRain_leafOff_mean = calInterception(LAI_leafOff_ar_mean,stormLeafOff,nlcd_tc)
    intercept_leafOff_up,   canRain_leafOff_up   = calInterception(LAI_leafOff_ar_up,stormLeafOff,nlcd_tc)


    # calculate total (leaf-off + leaf-on) values
    intercept_all_low = intercept_leafOn_low  + intercept_leafOff_low
    intercept_all_mean= intercept_leafOn_mean + intercept_leafOff_mean
    intercept_all_up  = intercept_leafOn_up   + intercept_leafOff_up

    canopyRain_all_low = canRain_leafOn_low   + canRain_leafOff_low
    canopyRain_all_mean= canRain_leafOn_mean  + canRain_leafOff_mean
    canopyRain_all_up  = canRain_leafOn_up    + canRain_leafOff_up

    # get the sum of arrays
    all_intercept_low = np.nansum(intercept_all_low)
    all_intercept_mean= np.nansum(intercept_all_mean)
    all_intercept_up  = np.nansum(intercept_all_up) 

    all_canopy_rain_low = np.nansum(canopyRain_all_low)
    all_canopy_rain_mean= np.nansum(canopyRain_all_mean)
    all_canopy_rain_up  = np.nansum(canopyRain_all_up)

    # write arrays in a dataframe

    df = pd.DataFrame({'landcover':nlcd_lc.flatten(),'landuse':nlud.flatten(),
                       'interception_low':intercept_all_low.flatten(),
                       'interception_mean':intercept_all_mean.flatten(),
                       'interception_up':intercept_all_up.flatten(),
                       'canopyRain_low': canopyRain_all_low.flatten(),
                       'canopyRain_mean':canopyRain_all_mean.flatten(),
                       'canopyRain_up': canopyRain_all_up.flatten()

                      })
    df=(df[df.interception_low>0])

    dfAgrLandcover=df.groupby(['landcover']).agg({'interception_low': ['sum'],'interception_mean':['sum'],'interception_up':['sum'],'canopyRain_mean':['sum']})
    dfAgrLanduse  =df.groupby(['landuse']).agg({'interception_low': ['sum'],'interception_mean':['sum'],'interception_up':['sum'],'canopyRain_mean':['sum']})
    dfAgrLandcover=pd.DataFrame(dfAgrLandcover.to_records())
    dfAgrLanduse  =pd.DataFrame(dfAgrLanduse.to_records())
    # set some fields
    dfAgrLandcover['GISJoin']=GISjoins
    dfAgrLandcover['eaa_id']=eaa_id
    dfAgrLandcover['Name']   = eaa_name
    dfAgrLandcover['CSO'] =CSO_Type
    # dfAgrLandcover['st_unique_ID']=station
    dfAgrLanduse['GISJoin']=GISjoins
    dfAgrLanduse['Name']   = eaa_name
    dfAgrLanduse['eaa_id']=eaa_id
    dfAgrLanduse['CSO'] =CSO_Type

    # dfAgrLanduse['st_unique_ID']=station
    lstDFsLandcover.append(dfAgrLandcover)
    lstDFsLanduse.append(dfAgrLanduse)
    print ('finshed working on ',eaa_name," - ",eaa_id)
#     except:
#         print ('got an error for ',eaa_name," - ",eaa_id)
#         lstErrorEAA.append([eaa_name,eaa_id])

C:\ProgramData\Anaconda2\envs\myenv\lib\site-packages\fiona\collection.py:331: FionaDeprecationWarning: Collection slicing is deprecated and will be disabled in a future version.
  return self.session.__getitem__(item)


finshed working on  Cambridge city  -  481


In [158]:
allDfsLandcover = pd.concat(lstDFsLandcover)
allDfsLanduse   = pd.concat(lstDFsLanduse)

In [129]:
allDfsLanduse.head(2)

,landuse,"('interception_low', 'sum')","('interception_mean', 'sum')","('interception_up', 'sum')","('canopyRain_mean', 'sum')",GISJoin,Name,eaa_id,CSO
0,111.0,4176.212059,13348.403250,14462.817275,123462.0972,G10,LiitleDry - Outside,10,0
1,113.0,918.366487,3029.695735,3282.635008,28022.2722,G10,LiitleDry - Outside,10,0


In [130]:
allDfsLanduse['LU_Label'] = allDfsLanduse['landuse'].apply(landUseCodeConvert)
allDfsLandcover['LC_Lable'] = allDfsLandcover['landcover'].apply(landCoverCodeConvert)
allDfsLanduse['NAICS_Code'] = allDfsLanduse['landuse'].apply(landuseToNAIOCS)

In [131]:
allDfsLanduse['Year'] = year
allDfsLanduse['LandCoverUse']='LandUse'
allDfsLandcover['Year'] = year
allDfsLandcover['LandCoverUse']='LandCover'

In [63]:
allDfsLandcover.to_csv(r'D:\OneDriveSyncFolder\OneDrive - The University of Colorado Denver\Ecosystem Service Accounting Project\Papers\Urban EA\Results\2020Version\Rainfall\Rainfall_allDfsLandcover_processe_{}_urbanAreas.csv'.format(year))
allDfsLanduse.to_csv(r'D:\OneDriveSyncFolder\OneDrive - The University of Colorado Denver\Ecosystem Service Accounting Project\Papers\Urban EA\Results\2020Version\Rainfall\Rainfall_allDfsLanduse_processed_{}_urbanAreas.csv'.format(year))

In [132]:
year

2016

In [134]:
folderAddress = r'D:\OneDriveSyncFolder\OneDrive - The University of Colorado Denver\Ecosystem Service Accounting Project\Papers\Urban EA\Results\2020Version\Rainfall\watershedsDenver'
allDfsLandcover.to_csv(folderAddress+r'\watershedRainfallDenver2016.csv')

In [161]:
all_intercept_mean.shape

()

In [133]:
allDfsLandcover["('interception_low', 'sum')"].sum()

1305328.7357919007

In [ ]:
print('minutes elapsed = ' ,(time.time()-t1)/60)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(all_intercept_mean)

In [41]:
addressHighResCity = r'D:\temp\UrbanEA_HighRes\Denver_tree_Cover.img'
with rasterio.open(addressHighResCity) as rsc:
    kwdsHighResCity = rsc.meta.copy()
    HighResBound=rsc.bounds
    HighResNoData=kwdsHighResCity['nodata']
    cellSizeHighRes= kwdsHighResCity['transform'][0]
    HighResWindow = findWindow(polyBound, HighResBound,cellSizeHighRes)
    HighResWindow_use=((HighResWindow[0],HighResWindow[1]),(HighResWindow[2],HighResWindow[3]))
    HighResCityAr=rsc.read(1, window=HighResWindow_use)
    HighResCityAr=HighResCityAr.astype('float')
    nlcd_tc_nodata = rsc.nodata
    HighResCityAr[HighResCityAr==nlcd_tc_nodata]=0
    nlcd_tc= HighResCityAr*9
    nlcd_tc[eaaAr!=eaa_id]=np.nan
        
# with rasterio.open(nlcd_tc_Address) as rst_nlcd_tc:
#     nlcd_tc_win_ar=rst_nlcd_tc.read(1, window=window_use)
#     nlcd_tc_win_ar=nlcd_tc_win_ar.astype('float')
#     # convert percent cover to area of tc
#     nlcd_tc= nlcd_tc_win_ar*9
#     nlcd_tc[eaaAr!=eaa_id]=np.nan

In [42]:
np.nanmean(nlcd_tc)

87.63124384932873